### Alpha Vantage Cash Flow Data Request

https://www.alphavantage.co/

In [1]:
# Import dependancies
import pandas as pd
import requests
import json
import sys
import time
import csv
import os

# Import the API key.
from config2 import stocks_p_api_key

In [2]:
company_overview_df = pd.read_csv('raw_dataset/company_overview.csv')
company_overview_df.head()

,SYMBOL,ASSET_TYPE,NAME,EXCHANGE,CURRENCY,COUNTRY,SECTOR,INDUSTRY,ADDRESS
0,TWCTU,Common Stock,TWC Tech Holdings II Corp,NASDAQ,USD,USA,NaN,NaN,"Four Embarcadero Center, San Francisco, CA, Un..."
1,SVACU,Common Stock,Starboard Value Acquisition Corp,NASDAQ,USD,USA,Financial Services,Shell Companies,"777 Third Avenue, New York, NY, United States,..."
2,ITACU,Common Stock,"Industrial Tech Acquisitions, Inc",NASDAQ,USD,USA,Financial Services,Shell Companies,"5090 Richmond Avenue, Houston, TX, United Stat..."
3,BCTG,Common Stock,BCTG Acquisition Corp,NASDAQ,USD,USA,Financial Services,Shell Companies,"11682 El Camino Real, San Diego, CA, United St..."
4,INAQU,Common Stock,INSU Acquisition Corp. II,NASDAQ,USD,USA,Financial Services,Shell Companies,"2929 Arch Street, Philadelphia, PA, United Sta..."


In [3]:
company_overview_df.shape

(2685, 9)

In [4]:
company_overview_df.dtypes

SYMBOL        object
ASSET_TYPE    object
NAME          object
EXCHANGE      object
CURRENCY      object
COUNTRY       object
SECTOR        object
INDUSTRY      object
ADDRESS       object
dtype: object

In [5]:
# Create a list of IPO symbols from the company_overview_df
IPO_list = company_overview_df['SYMBOL'].tolist()
IPO_list

['TWCTU',
 'SVACU',
 'ITACU',
 'BCTG',
 'INAQU',
 'CAPAU',
 'PRFX',
 'AUVI',
 'HCDI',
 'BTAQU',
 'CFIIU',
 'XPEV',
 'FTOCU',
 'KYMR',
 'NNOX',
 'HRMY',
 'CVAC',
 'LCAPU',
 'BEKE',
 'NTST',
 'ARYA',
 'CMPI',
 'FRLN',
 'IBEX',
 'VMACU',
 'GRSVU',
 'KSMTU',
 'AFIB',
 'OSH',
 'RKT',
 'BIGC',
 'RXT',
 'BOWXU',
 'HOLUU',
 'HSAQ',
 'GOED',
 'NHICU',
 'ETACU',
 'FTHM',
 'VITL',
 'ALVR',
 'LI',
 'VERX',
 'ACEVU',
 'ANNX',
 'GRCYU',
 'INZY',
 'ITOS',
 'NRIX',
 'ERESU',
 'MEG',
 'EDTK',
 'JAMF',
 'PSACU',
 'ALXO',
 'BLI',
 'DFHTU',
 'PAND',
 'RLAY',
 'TIG',
 'GOCO',
 'NCNO',
 'NKTX',
 'PSTX',
 'QH',
 'BLCT',
 'LMND',
 'DNB',
 'API',
 'AKUS',
 'ACI',
 'EBON',
 'FUSN',
 'PYPD',
 'BRLIU',
 'FMTX',
 'GTH',
 'PROG',
 'RPTX',
 'RPRX',
 'AZEK',
 'BNR',
 'GBIO',
 'PCVX',
 'LTRN',
 'UCL',
 'HECCU',
 'VRM',
 'AMTI',
 'ARYBU',
 'DADA',
 'LEGN',
 'MCACU',
 'FOUR',
 'ZI',
 'PLRX',
 'SLQT',
 'NOVSU',
 'AYLA',
 'CLEU',
 'KC',
 'CGROU',
 'LYRA',
 'ORIC',
 'KROS',
 'ZNTL',
 'WIMI',
 'IMRA',
 'DFPHU',
 'GFL',
 'PA

In [6]:
# Check length of IPO_list
len(IPO_list)

2685

### Request Cash Flow from Alpha Vantage

Create a loop to retreive Cash Flow for each symbol in the IPO_list

In [7]:
# Create For loop to fetch responses from Alpha Vantage

KEY_ERROR_List = []
index = 0

for symbol in IPO_list:
    
    # Define url
    base_income_statement_url = "https://www.alphavantage.co/query?function=CASH_FLOW&symbol="    
    
    Q_fiscalDateEnding_List = []
    Q_reportedCurrency_List = []
    Q_operatingCashflow_List = []
    Q_paymentsForOperatingActivities_List = []
    Q_proceedsFromOperatingActivities_List = []
    Q_changeInOperatingLiabilities_List = []
    Q_changeInOperatingAssets_List = []
    Q_depreciationDepletionAndAmortization_List = []
    Q_capitalExpenditures_List = []
    Q_changeInReceivables_List = []
    Q_changeInInventory_List = []
    Q_profitLoss_List = []
    Q_cashflowFromInvestment_List = []
    Q_cashflowFromFinancing_List = []
    Q_proceedsFromRepaymentsOfShortTermDebt_List = []
    Q_paymentsForRepurchaseOfCommonStock_List = []
    Q_paymentsForRepurchaseOfEquity_List = []
    Q_paymentsForRepurchaseOfPreferredStock_List = []
    Q_dividendPayout_List = []
    Q_dividendPayoutCommonStock_List = []
    Q_dividendPayoutPreferredStock_List = []
    Q_proceedsFromIssuanceOfCommonStock_List = []
    Q_proceedsFromIssuanceOfLongTermDebtAndCapitalSecuritiesNet_List = []
    Q_proceedsFromIssuanceOfPreferredStock_List = []
    Q_proceedsFromRepurchaseOfEquity_List = []
    Q_proceedsFromSaleOfTreasuryStock_List = []
    Q_changeInCashAndCashEquivalents_List = []
    Q_changeInExchangeRate_List = []
    Q_netIncome_List = []

    
    symbol_parameter = symbol

    full_url = f'{base_income_statement_url}{symbol_parameter}&apikey={stocks_p_api_key}'

    # Make an API call to the API and get the JSON object
    response = requests.get(full_url).json()

    try:
    
        for i in range(len(response['quarterlyReports'])):

            Q_fiscalDateEnding_response = response['quarterlyReports'][i]['fiscalDateEnding']
            Q_reportedCurrency_response = response['quarterlyReports'][i]['reportedCurrency']
            Q_operatingCashflow_response = response['quarterlyReports'][i]['operatingCashflow']
            Q_paymentsForOperatingActivities_response = response['quarterlyReports'][i]['paymentsForOperatingActivities']
            Q_proceedsFromOperatingActivities_response = response['quarterlyReports'][i]['proceedsFromOperatingActivities']
            Q_changeInOperatingLiabilities_response = response['quarterlyReports'][i]['changeInOperatingLiabilities']
            Q_changeInOperatingAssets_response = response['quarterlyReports'][i]['changeInOperatingAssets']
            Q_depreciationDepletionAndAmortization_response = response['quarterlyReports'][i]['depreciationDepletionAndAmortization']
            Q_capitalExpenditures_response = response['quarterlyReports'][i]['capitalExpenditures']
            Q_changeInReceivables_response = response['quarterlyReports'][i]['changeInReceivables']
            Q_changeInInventory_response = response['quarterlyReports'][i]['changeInInventory']
            Q_profitLoss_response = response['quarterlyReports'][i]['profitLoss']
            Q_cashflowFromInvestment_response = response['quarterlyReports'][i]['cashflowFromInvestment']
            Q_cashflowFromFinancing_response = response['quarterlyReports'][i]['cashflowFromFinancing']
            Q_proceedsFromRepaymentsOfShortTermDebt_response = response['quarterlyReports'][i]['proceedsFromRepaymentsOfShortTermDebt']
            Q_paymentsForRepurchaseOfCommonStock_response = response['quarterlyReports'][i]['paymentsForRepurchaseOfCommonStock']
            Q_paymentsForRepurchaseOfEquity_response = response['quarterlyReports'][i]['paymentsForRepurchaseOfEquity']
            Q_paymentsForRepurchaseOfPreferredStock_response = response['quarterlyReports'][i]['paymentsForRepurchaseOfPreferredStock']
            Q_dividendPayout_response = response['quarterlyReports'][i]['dividendPayout']
            Q_dividendPayoutCommonStock_response = response['quarterlyReports'][i]['dividendPayoutCommonStock']
            Q_dividendPayoutPreferredStock_response = response['quarterlyReports'][i]['dividendPayoutPreferredStock']
            Q_proceedsFromIssuanceOfCommonStock_response = response['quarterlyReports'][i]['proceedsFromIssuanceOfCommonStock']
            Q_proceedsFromIssuanceOfLongTermDebtAndCapitalSecuritiesNet_response = response['quarterlyReports'][i]['proceedsFromIssuanceOfLongTermDebtAndCapitalSecuritiesNet']
            Q_proceedsFromIssuanceOfPreferredStock_response = response['quarterlyReports'][i]['proceedsFromIssuanceOfPreferredStock']
            Q_proceedsFromRepurchaseOfEquity_response = response['quarterlyReports'][i]['proceedsFromRepurchaseOfEquity']
            Q_proceedsFromSaleOfTreasuryStock_response = response['quarterlyReports'][i]['proceedsFromSaleOfTreasuryStock']
            Q_changeInCashAndCashEquivalents_response = response['quarterlyReports'][i]['changeInCashAndCashEquivalents']
            Q_changeInExchangeRate_response = response['quarterlyReports'][i]['changeInExchangeRate']
            Q_netIncome_response = response['quarterlyReports'][i]['netIncome']


            Q_fiscalDateEnding_List.append(Q_fiscalDateEnding_response)
            Q_reportedCurrency_List.append(Q_reportedCurrency_response)
            Q_operatingCashflow_List.append(Q_operatingCashflow_response)
            Q_paymentsForOperatingActivities_List.append(Q_paymentsForOperatingActivities_response)
            Q_proceedsFromOperatingActivities_List.append(Q_proceedsFromOperatingActivities_response)
            Q_changeInOperatingLiabilities_List.append(Q_changeInOperatingLiabilities_response)
            Q_changeInOperatingAssets_List.append(Q_changeInOperatingAssets_response)
            Q_depreciationDepletionAndAmortization_List.append(Q_depreciationDepletionAndAmortization_response)
            Q_capitalExpenditures_List.append(Q_capitalExpenditures_response)
            Q_changeInReceivables_List.append(Q_changeInReceivables_response)
            Q_changeInInventory_List.append(Q_changeInInventory_response)
            Q_profitLoss_List.append(Q_profitLoss_response)
            Q_cashflowFromInvestment_List.append(Q_cashflowFromInvestment_response)
            Q_cashflowFromFinancing_List.append(Q_cashflowFromFinancing_response)
            Q_proceedsFromRepaymentsOfShortTermDebt_List.append(Q_proceedsFromRepaymentsOfShortTermDebt_response)
            Q_paymentsForRepurchaseOfCommonStock_List.append(Q_paymentsForRepurchaseOfCommonStock_response)
            Q_paymentsForRepurchaseOfEquity_List.append(Q_paymentsForRepurchaseOfEquity_response)
            Q_paymentsForRepurchaseOfPreferredStock_List.append(Q_paymentsForRepurchaseOfPreferredStock_response)
            Q_dividendPayout_List.append(Q_dividendPayout_response)
            Q_dividendPayoutCommonStock_List.append(Q_dividendPayoutCommonStock_response)
            Q_dividendPayoutPreferredStock_List.append(Q_dividendPayoutPreferredStock_response)
            Q_proceedsFromIssuanceOfCommonStock_List.append(Q_proceedsFromIssuanceOfCommonStock_response)
            Q_proceedsFromIssuanceOfLongTermDebtAndCapitalSecuritiesNet_List.append(Q_proceedsFromIssuanceOfLongTermDebtAndCapitalSecuritiesNet_response)
            Q_proceedsFromIssuanceOfPreferredStock_List.append(Q_proceedsFromIssuanceOfPreferredStock_response)
            Q_proceedsFromRepurchaseOfEquity_List.append(Q_proceedsFromRepurchaseOfEquity_response)
            Q_proceedsFromSaleOfTreasuryStock_List.append(Q_proceedsFromSaleOfTreasuryStock_response)
            Q_changeInCashAndCashEquivalents_List.append(Q_changeInCashAndCashEquivalents_response)
            Q_changeInExchangeRate_List.append(Q_changeInExchangeRate_response)
            Q_netIncome_List.append(Q_netIncome_response)


        # Transform each list into a series

        Q_fiscalDateEnding_Series = pd.Series(Q_fiscalDateEnding_List)
        Q_reportedCurrency_Series = pd.Series(Q_reportedCurrency_List)
        Q_operatingCashflow_Series = pd.Series(Q_operatingCashflow_List)
        Q_paymentsForOperatingActivities_Series = pd.Series(Q_paymentsForOperatingActivities_List)
        Q_proceedsFromOperatingActivities_Series = pd.Series(Q_proceedsFromOperatingActivities_List)
        Q_changeInOperatingLiabilities_Series = pd.Series(Q_changeInOperatingLiabilities_List)
        Q_changeInOperatingAssets_Series = pd.Series(Q_changeInOperatingAssets_List)
        Q_depreciationDepletionAndAmortization_Series = pd.Series(Q_depreciationDepletionAndAmortization_List)
        Q_capitalExpenditures_Series = pd.Series(Q_capitalExpenditures_List)
        Q_changeInReceivables_Series = pd.Series(Q_changeInReceivables_List)
        Q_changeInInventory_Series = pd.Series(Q_changeInInventory_List)
        Q_profitLoss_Series = pd.Series(Q_profitLoss_List)
        Q_cashflowFromInvestment_Series = pd.Series(Q_cashflowFromInvestment_List)
        Q_cashflowFromFinancing_Series = pd.Series(Q_cashflowFromFinancing_List)
        Q_proceedsFromRepaymentsOfShortTermDebt_Series = pd.Series(Q_proceedsFromRepaymentsOfShortTermDebt_List)
        Q_paymentsForRepurchaseOfCommonStock_Series = pd.Series(Q_paymentsForRepurchaseOfCommonStock_List)
        Q_paymentsForRepurchaseOfEquity_Series = pd.Series(Q_paymentsForRepurchaseOfEquity_List)
        Q_paymentsForRepurchaseOfPreferredStock_Series = pd.Series(Q_paymentsForRepurchaseOfPreferredStock_List)
        Q_dividendPayout_Series = pd.Series(Q_dividendPayout_List)
        Q_dividendPayoutCommonStock_Series = pd.Series(Q_dividendPayoutCommonStock_List)
        Q_dividendPayoutPreferredStock_Series = pd.Series(Q_dividendPayoutPreferredStock_List)
        Q_proceedsFromIssuanceOfCommonStock_Series = pd.Series(Q_proceedsFromIssuanceOfCommonStock_List)
        Q_proceedsFromIssuanceOfLongTermDebtAndCapitalSecuritiesNet_Series = pd.Series(Q_proceedsFromIssuanceOfLongTermDebtAndCapitalSecuritiesNet_List)
        Q_proceedsFromIssuanceOfPreferredStock_Series = pd.Series(Q_proceedsFromIssuanceOfPreferredStock_List)
        Q_proceedsFromRepurchaseOfEquity_Series = pd.Series(Q_proceedsFromRepurchaseOfEquity_List)
        Q_proceedsFromSaleOfTreasuryStock_Series = pd.Series(Q_proceedsFromSaleOfTreasuryStock_List)
        Q_changeInCashAndCashEquivalents_Series = pd.Series(Q_changeInCashAndCashEquivalents_List)
        Q_changeInExchangeRate_Series = pd.Series(Q_changeInExchangeRate_List)
        Q_netIncome_Series = pd.Series(Q_netIncome_List)

        frame = {
            'SYMBOL':symbol_parameter,
            'fiscalDateEnding' : Q_fiscalDateEnding_Series,
            'reportedCurrency' : Q_reportedCurrency_Series,
            'operatingCashflow' : Q_operatingCashflow_Series,
            'paymentsForOperatingActivities' : Q_paymentsForOperatingActivities_Series,
            'proceedsFromOperatingActivities' : Q_proceedsFromOperatingActivities_Series,
            'changeInOperatingLiabilities' : Q_changeInOperatingLiabilities_Series,
            'changeInOperatingAssets' : Q_changeInOperatingAssets_Series,
            'depreciationDepletionAndAmortization' : Q_depreciationDepletionAndAmortization_Series,
            'capitalExpenditures' : Q_capitalExpenditures_Series,
            'changeInReceivables' : Q_changeInReceivables_Series,
            'changeInInventory' : Q_changeInInventory_Series,
            'profitLoss' : Q_profitLoss_Series,
            'cashflowFromInvestment' : Q_cashflowFromInvestment_Series,
            'cashflowFromFinancing' : Q_cashflowFromFinancing_Series,
            'proceedsFromRepaymentsOfShortTermDebt' : Q_proceedsFromRepaymentsOfShortTermDebt_Series,
            'paymentsForRepurchaseOfCommonStock' : Q_paymentsForRepurchaseOfCommonStock_Series,
            'paymentsForRepurchaseOfEquity' : Q_paymentsForRepurchaseOfEquity_Series,
            'paymentsForRepurchaseOfPreferredStock' : Q_paymentsForRepurchaseOfPreferredStock_Series,
            'dividendPayout' : Q_dividendPayout_Series,
            'dividendPayoutCommonStock' : Q_dividendPayoutCommonStock_Series,
            'dividendPayoutPreferredStock' : Q_dividendPayoutPreferredStock_Series,
            'proceedsFromIssuanceOfCommonStock' : Q_proceedsFromIssuanceOfCommonStock_Series,
            'proceedsFromIssuanceOfLongTermDebtAndCapitalSecuritiesNet' : Q_proceedsFromIssuanceOfLongTermDebtAndCapitalSecuritiesNet_Series,
            'proceedsFromIssuanceOfPreferredStock' : Q_proceedsFromIssuanceOfPreferredStock_Series,
            'proceedsFromRepurchaseOfEquity' : Q_proceedsFromRepurchaseOfEquity_Series,
            'proceedsFromSaleOfTreasuryStock' : Q_proceedsFromSaleOfTreasuryStock_Series,
            'changeInCashAndCashEquivalents' : Q_changeInCashAndCashEquivalents_Series,
            'changeInExchangeRate' : Q_changeInExchangeRate_Series,
            'netIncome' : Q_netIncome_Series
        }

        q_cash_flow_df = pd.DataFrame(frame)

        # Exporting test income statement dataframe to csv file
        save_file_path = f'raw_dataset/cash_flow/{symbol}_cash_flow.csv'
        q_cash_flow_df.to_csv(save_file_path,index=False)
        
    except KeyError:
        print(f"KeyError occured on {symbol}")
        KEY_ERROR_List.append(symbol)
        
    print(index, symbol)
    index += 1

print(len(ERROR_List))

C:\Users\User\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:117: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
C:\Users\User\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:118: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
C:\Users\User\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:119: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
C:\Users\User\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:120: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warni

0 TWCTU
1 SVACU
2 ITACU
3 BCTG
4 INAQU
5 CAPAU
6 PRFX
7 AUVI
8 HCDI
9 BTAQU
10 CFIIU
11 XPEV
12 FTOCU
13 KYMR
14 NNOX
15 HRMY
16 CVAC
17 LCAPU
18 BEKE
19 NTST
20 ARYA
21 CMPI
22 FRLN
23 IBEX
24 VMACU
25 GRSVU
26 KSMTU
27 AFIB
28 OSH
29 RKT
30 BIGC
31 RXT
32 BOWXU
33 HOLUU
34 HSAQ
35 GOED
36 NHICU
37 ETACU
38 FTHM
39 VITL
40 ALVR
41 LI
42 VERX
43 ACEVU
44 ANNX
45 GRCYU
46 INZY
47 ITOS
48 NRIX
49 ERESU
50 MEG
51 EDTK
52 JAMF
53 PSACU
54 ALXO
55 BLI
56 DFHTU
57 PAND
58 RLAY
59 TIG
60 GOCO
61 NCNO
62 NKTX
63 PSTX
64 QH
65 BLCT
66 LMND
67 DNB
68 API
69 AKUS
70 ACI
71 EBON
72 FUSN
73 PYPD
74 BRLIU
75 FMTX
76 GTH
77 PROG
78 RPTX
79 RPRX
80 AZEK
81 BNR
82 GBIO
83 PCVX
84 LTRN
85 UCL
86 HECCU
87 VRM
88 AMTI
89 ARYBU
90 DADA
91 LEGN
92 MCACU
93 FOUR
94 ZI
95 PLRX
96 SLQT
97 NOVSU
98 AYLA
99 CLEU
100 KC
101 CGROU
102 LYRA
103 ORIC
104 KROS
105 ZNTL
106 WIMI
107 IMRA
108 DFPHU
109 GFL
110 PASG
111 ZCMD
112 ESSCU
113 NBACU
114 ZGYHU
115 GRIL
116 RVMD
117 HUIZ
118 GNRSU
119 NREF
120 ONEW
121 PFHD
12

930 SKIS
931 CNV
932 HABT
933 NEOT
934 PGRE
935 EHIC
936 FGEN
937 LMRK
938 NDRM
939 VA
940 NAP
941 PRAH
942 SKYS
943 AXTA
944 JYNT
945 FRPT
946 INCR
947 TBK
948 CHRS
949 NVRO
950 UPLD
951 AM
952 XENE
953 BOOT
954 SHLX
955 SIEN
956 HDRAU
957 ZAYO
958 ATRA
959 FWP
960 GWB
961 DPLO
962 HUBS
963 OMAM
964 USDP
965 VBTX
966 DERM
967 FMSA
968 YDLE
969 AAC
970 ATTO
971 CALA
972 JPEP
973 VWR
974 W
975 CADTU
976 VBLT
977 VSLR
978 CNNX
979 TVPT
980 CYBR
981 MDLY
982 SFS
983 VTAE
984 BABA
985 FOMX
986 PRQR
987 CIVI
988 TKAI
989 AFMD
990 RWLK
991 SMACU
992 BNK
993 GNBC
994 ICD
995 RYI
996 HMLP
997 DSKY
998 TTOO
999 EARS
1000 FCB
1001 LOXO
1002 MBLY
1003 VTTI
1004 CTLT
1005 HQY
1006 MCUR
1007 MRNS
1008 SYF
1009 RIGP
1010 WLKP
1011 SPKE
1012 WMS
1013 LOCO
1014 INNL
1015 OCUL
1016 OEC
1017 IMDZ
1018 XENT
1019 PFNX
1020 TSQ
1021 MTBC
1022 GLOB
1023 SAGE
1024 TERP
1025 TRUP
1026 TUBE
1027 CDNA
1028 ROKA
1029 TRTLU
1030 TFSCU
1031 IRMD
1032 ISTR
1033 NERV
1034 MIK
1035 GGACU
1036 GPRO
1037 TCPI
1038 IMPR

1785 LL
1786 QELP
1787 ENSG
1788 OSP
1789 NPD
1790 APEI
1791 ICXT
1792 GRO
1793 BFRM
1794 SD
1795 PROJ
1796 SDBT
1797 GA
1798 GXDX
1799 PMII
1800 FGXI
1801 PZN
1802 ULTA
1803 VNR
1804 LFT
1805 CVI
1806 DFT
1807 NED
1808 VM
1809 TARG
1810 TGH
1811 CML
1812 MAPP
1813 STV
1814 CTCT
1815 DUF
1816 FLY
1817 ATHN
1818 ENP
1819 VMW
1820 PRGN
1821 ZINC
1822 MELI
1823 DMAN
1824 WX
1825 HIRE
1826 EJ
1827 MASI
1828 SCMP
1829 VRTU
1830 CXO
1831 G
1832 LULU
1833 VOLT
1834 PWRD
1835 TYPE
1836 REXX
1837 VR
1838 BLOG
1839 OWW
1840 AIRV
1841 HGG
1842 NZ
1843 LIMC
1844 DHX
1845 EBTX
1846 SHOR
1847 PPO
1848 PRO
1849 AUTH
1850 SPRD
1851 SCOR
1852 DDUP
1853 SEP
1854 CRE
1855 BX
1856 BWY
1857 YGE
1858 LLNW
1859 RGDX
1860 INFN
1861 STAR
1862 JAZZ
1863 FOLD
1864 CLNE
1865 GLRE
1866 BGS
1867 TRS
1868 ENOC
1869 TTGT
1870 EURX
1871 CSUN
1872 CAP
1873 CLR
1874 SKH
1875 PINN
1876 PODD
1877 JMP
1878 ACM
1879 ATV
1880 CAVM
1881 EDN
1882 OREX
1883 CNK
1884 SCR
1885 PCS
1886 COMV
1887 FSR
1888 CPLP
1889 GSIT
1890 SENO


2638 VASC
2639 AGNT
2640 SPRT
2641 OVTI
2642 IMNY
2643 TNSI
2644 SOHU
2645 ACLS
2646 ENTG
2647 TKC
2648 NTES
2649 MBT
2650 EXFO
2651 PRSE
2652 CARE
2653 CPST
2654 VRGE
2655 CKCM
2656 MRVL
2657 STLW
2658 CRL
2659 ACCL
2660 CKSW
2661 CHU
2662 CPHD
2663 HAND
2664 DNDN
2665 OSCA
2666 AUGT
2667 REDF
2668 ISRG
2669 CYH
2670 UPCS
2671 CWLD
2672 ONIS
2673 NRG
2674 SMDI
2675 SONS
2676 CTLM
2677 ICST
2678 CSAV
2679 PXLW
2680 PRTH
2681 NGTC
2682 IBEM
2683 NUFO
2684 SQSW


NameError: name 'ERROR_List' is not defined

In [8]:
# Print length of Key Error List
len(KEY_ERROR_List)

0